<a href="https://colab.research.google.com/github/stefkong1982/netology.ru/blob/Master/%D0%91%D0%B8%D0%B1%D0%BB%D0%B8%D0%BE%D1%82%D0%B5%D0%BA%D0%B8%20Python%20%D0%B4%D0%BB%D1%8F%20%D0%B0%D0%BD%D0%B0%D0%BB%D0%B8%D0%B7%D0%B0%20%D0%B4%D0%B0%D0%BD%D0%BD%D1%8B%D1%85/%D0%9B%D0%B5%D0%BA%D1%86%D0%B8%D1%8F%20%D0%9E%D1%81%D0%BD%D0%BE%D0%B2%D1%8B%20%D0%B2%D0%B5%D0%B1-%D1%81%D0%BA%D1%80%D0%B0%D0%BF%D0%B8%D0%BD%D0%B3%D0%B0/%D0%94%D0%97_%D0%9A%D0%BE%D0%BD%D0%B4%D1%80%D0%B0%D1%82%D1%8C%D0%B5%D0%B2_NUMPY-48%20%D0%BA%20%D0%BB%D0%B5%D0%BA%D1%86%D0%B8%D0%B8%20%D0%9E%D1%81%D0%BD%D0%BE%D0%B2%D1%8B%20%D0%B2%D0%B5%D0%B1-%D1%81%D0%BA%D1%80%D0%B0%D0%BF%D0%B8%D0%BD%D0%B3%D0%B0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Домашнее задание к лекции "Основы веб-скрапинга"

## Обязательная часть

Вам необходимо написать функцию, которая будет основана на **поиске** по сайту [habr.com](https://habr.com/ru/search/).
Функция в качестве параметра должна принимать **список** запросов для поиска (например, `['python', 'анализ данных']`) и на основе материалов, попавших в результаты поиска по **каждому** запросу, возвращать датафрейм вида:

```
<дата> - <заголовок> - <ссылка на материал>
```

В рамках задания предполагается работа только с одной (первой) страницей результатов поисковой выдачи для каждого запроса. Материалы в датафрейме не должны дублироваться, если они попадали в результаты поиска для нескольких запросов из списка.


In [30]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import time
import urllib.parse

base_url = 'https://habr.com'

In [2]:
res = requests.get('https://habr.com/ru/search/?q=анализ%20данных&target_type=posts&order=relevance')
res

<Response [200]>

In [ ]:
soup = BeautifulSoup(res.text)
soup

In [ ]:
news = soup.find_all('article', class_='tm-articles-list__item')
print(len(news))
print(news[0])

In [ ]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import time
import urllib.parse

base_url = 'https://habr.com'
res = requests.get('https://habr.com/ru/search/?q=анализ%20данных&target_type=posts&order=relevance')
soup = BeautifulSoup(res.text)
news = soup.find_all('article', class_='tm-articles-list__item')

for article in news:
    # Проверка наличия заголовка в оригинальной разметке
    title_element = article.find('h2', class_='tm-megapost-snippet__title')
    if title_element is not None:
        title = title_element.text.strip()
        print("Заголовок:", title)
    else:
        # Проверка наличия заголовка в другой разметке
        title_element_alt = article.find('h2', class_='tm-title tm-title_h2')
        if title_element_alt is not None:
            title = title_element_alt.find('span').text.strip()
            print("Заголовок:", title)
        else:
            print("Заголовок не найден")

    # Проверка наличия ссылки в оригинальной разметке
    link_element = article.find('a', class_='tm-megapost-snippet__link tm-megapost-snippet__date')
    if link_element is not None:
        relative_link = link_element.get('href')
        full_link = urllib.parse.urljoin(base_url, relative_link)
        print(full_link)
    else:
        # Проверка наличия ссылки в другой разметке
        link_element_alt = article.find('h2', class_='tm-title tm-title_h2').find('a', class_='tm-title__link')
        if link_element_alt is not None:
            relative_link = link_element_alt.get('href')
            full_link = urllib.parse.urljoin(base_url, relative_link)
            print(full_link)
        else:
            print("Ссылка не найдена")

    # Проверка наличия даты в оригинальной разметке
    date_element = article.find('time', class_='tm-megapost-snippet__datetime-published')
    if date_element is not None:
        date = date_element.get('title')
        print("Дата:", date)
    else:
        # Проверка наличия даты в другой разметке
        date_element_alt = article.find('span', class_='tm-article-datetime-published').find('time')
        if date_element_alt is not None:
            date = date_element_alt.get('title')
            print("Дата:", date)
        else:
            print("Дата не найдена")

    print()


In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import urllib.parse
import time

def get_habr_posts(query):
    """
    Получает заголовки и ссылки на статьи с сайта habr.com по заданному запросу.

    Аргументы:
    query -- список запросов для поиска статей (например, ['python', 'анализ данных'])

    Возвращает:
    DataFrame -- датафрейм с информацией о заголовках и ссылках на статьи
    """

    habr_blog = pd.DataFrame()
    base_url = "https://habr.com"

    for q in query:
        URL = 'https://habr.com/ru/search/'
        params = {
            'q': q
        }
        req = requests.get(URL, params=params)
        time.sleep(0.3)
        soup = BeautifulSoup(req.text)
        articles = soup.find_all('article', class_='tm-articles-list__item')

        if not articles:
            continue

        posts = []
        for article in articles:
            title_element = article.find('h2', class_='tm-megapost-snippet__title')
            if title_element is not None:
                title = title_element.text.strip()
                link_element = article.find('a', class_='tm-megapost-snippet__link tm-megapost-snippet__date')
                relative_link = link_element.get('href')
                full_link = urllib.parse.urljoin(base_url, relative_link)
                date_element = article.find('time', class_='tm-megapost-snippet__datetime-published')
                date = date_element.get('title')
            else:
                title_element_alt = article.find('h2', class_='tm-title tm-title_h2')
                title = title_element_alt.find('span').text.strip()
                link_element_alt = article.find('h2', class_='tm-title tm-title_h2').find('a', class_='tm-title__link')
                relative_link = link_element_alt.get('href')
                full_link = urllib.parse.urljoin(base_url, relative_link)
                date_element_alt = article.find('span', class_='tm-article-datetime-published').find('time')
                date = date_element_alt.get('title')

            post = {'date': date, 'title': title, 'link': full_link}
            posts.append(post)


        habr_blog = pd.concat([habr_blog, pd.DataFrame(posts)], ignore_index=True)

    # Удаление дубликатов
    habr_blog = habr_blog.drop_duplicates()

    return habr_blog

# Пример использования функции
res = get_habr_posts(['python', 'анализ данных'])
res

## Дополнительная часть (необязательная)

Функция из обязательной части задания должна быть расширена следующим образом:
- кроме списка ключевых слов для поиска необходимо объявить параметр с количеством страниц поисковой выдачи. Т.е. при передаче в функцию аргумента `4` необходимо получить материалы с первых 4 страниц результатов;
- в датафрейме должны быть столбцы с полным текстом найденных материалов и количеством лайков:
```
<дата> - <заголовок> - <ссылка на материал> - <текст материала> - <количество лайков>
```
